### <center>Manage the Conversation History</center>

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.


'trim_messages'helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [2]:
GROQ_API_KEY="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"
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

## Calling our model
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.3-70b-specdec", groq_api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10c3f60d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10c409390>, model_name='llama-3.3-70b-specdec', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import AIMessage

In [8]:
from langchain_core.messages import SystemMessage,trim_messages
from langchain_core.messages import HumanMessage
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I am Surya Vishal"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like Buttorscoth ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage (content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [11]:
## 
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a Helpful assistant. Answer all the questions of the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [16]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
#from langchain_core.messages import HumanMessage

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages"))
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="Nenu Evaru?")],
        "languages": "Telugu"
    }
)

response.content

"Nuvvu Surya Vishal, nenu un assistant, chat chestunnaanu! (You're Surya Vishal, and I'm an assistant, chatting with you!)"

In [13]:
#HumanMessage(content="whats 2 + 2"),
#    AIMessage(content="4"),

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math prolem I did?")],
        "languages": "Tenglish"
    }
)
response.content

'You did a simple addition problem: 2 + 2.'

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [22]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config={"configurable":{"session_id":"chat5"}}

In [23]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my name?")],
        "languages": "English"
    },
    config=config
)

response.content

'Your name is Surya Vishal.'